In [19]:
import tqdm
import numpy as np
import pandas as pd

In [3]:
# add new features
def baseline_features(df):
    df['open_sub_close'] = df['Open'] - df['Close']
    df['high_div_low'] = df['High'] / df['Low']
    
    # MA8, 21, 50, 200 - close price
    df['close_ma8'] = df['Close'].rolling(8).mean()
    df['close_ma21'] = df['Close'].rolling(21).mean()
    df['close_ma50'] = df['Close'].rolling(50).mean()
    df['close_ma200'] = df['Close'].rolling(200).mean()
    
    # MA8, 21, 50, 200 - volume
    df['vol_ma8'] = df['Volume'].rolling(8).mean()
    df['vol_ma21'] = df['Volume'].rolling(21).mean()
    df['vol_ma50'] = df['Volume'].rolling(50).mean()
    df['vol_ma200'] = df['Volume'].rolling(200).mean()
    
    # RSI
    delta = df['Close'].diff()
    up, down = delta.clip(lower=0), -1*delta.clip(upper=0)
    ema_up, ema_down = up.ewm(com=13, adjust=False).mean(), down.ewm(com=13, adjust=False).mean()
    rs = ema_up / ema_down
    df['RSI'] = 100 - (100/(1+rs))
    
    # MACD
    exp1, exp2 = df['Close'].ewm(span=12, adjust=False).mean(), df['Close'].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    df['MACD'] = macd.ewm(span=9, adjust=False).mean()
    
    # OBV
    df['OBV'] = (np.sign(df['Close'].diff())*df['Volume']).fillna(0).cumsum()
    
    return df